In [7]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression

In [9]:
class ResidualUnit(keras.layers.layer):
    def __init__(self,filters,strides=1,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.activation=keras.activation.get(activation)
        self.main_layers=[
            keras.layers.Conv2D(filters,3,strides=strides,padding="same",use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters,3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()           
        ]
        self.skip_layers=[]
        if strides>1:
            self.skip_layers=[
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()
            ]
            
    def call(self,inputs):
        Z=inputs
        for layer in self.main_layers:
            Z=layer(Z)
        skip_Z=inputs
        for layer in self.skip_layers:
            skip_Z=layer(skip_Z)
        return self.activation(Z+skip_Z)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'layer'

In [3]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[224,224,3],padding="same",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2,padding="same"))
pre_filters=64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    
    if filters==prev_filters:
        strides=1
    else:
        strides=2
        model.add(ResidualUnit(filters,strides=strides))
        prev_filters=filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation="softmax"))

NameError: name 'prev_filters' is not defined